In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# ResNet V2

In [7]:
def conv2d_bn(x, filters, kernel_size, strides=1, padding='same'):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding=padding)(x)
    x = BatchNormalization()(x)
    return x

def residual_block(x, filters, kernel_size, strides=1):
    shortcut = x
    x = conv2d_bn(x, filters, kernel_size, strides)
    x = ReLU()(x)
    x = conv2d_bn(x, filters, kernel_size, 1)
    if x.shape != shortcut.shape:
        shortcut = Conv2D(filters=filters,
                          kernel_size=1,
                          strides=strides,
                          padding='same')(shortcut)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def resnetv2(input_shape, num_classes, num_layers, use_bottleneck=False):
    num_blocks = (num_layers - 2) // 9
    filters = 16

    inputs = Input(shape=input_shape)

    x = conv2d_bn(inputs, filters, 3)

    for i in range(num_blocks):
        for j in range(3):
            strides = 1
            if j == 0:
                strides = 2
            x = residual_block(x, filters, 3, strides)
        filters *= 2

    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)

    return model

model = resnetv2(input_shape=(224,224,3),  num_classes=200, num_layers=20, use_bottleneck=True)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_236 (Conv2D)            (None, 224, 224, 16  448         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_155 (Batch  (None, 224, 224, 16  64         ['conv2d_236[0][0]']             
 Normalization)                 )                                                           